In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [3]:
cityfile = "../WeatherPy/Output/cityweather.csv"

In [5]:
citydata = pd.read_csv(cityfile)
citydata

,City Number,City,Country,Lat,Long,Humidity,Temperature,Max Temp,Cloudiness,Wind Speed
0,0,Saint Paul Harbor,US,57.79,-152.41,75,44.60,44.60,90,6.93
1,1,Khatanga,RU,71.97,102.50,85,7.18,7.18,50,8.79
2,2,Ushuaia,AR,-54.80,-68.30,75,42.80,42.80,75,28.86
3,3,Saskylakh,RU,71.92,114.08,97,20.79,20.79,100,6.51
4,4,Port Blair,IN,11.67,92.75,83,80.53,80.53,100,14.67
...,...,...,...,...,...,...,...,...,...,...
548,548,La Asuncion,VE,11.03,-63.86,78,78.80,78.80,0,6.93
549,549,Rodrigues Alves,BR,-7.74,-72.65,40,92.68,92.68,23,4.99
550,550,Bhaderwah,IN,32.98,75.72,41,53.82,53.82,0,2.48
551,551,Lashio,MM,22.93,97.75,98,65.07,65.07,100,0.63


In [6]:
gmaps.configure(api_key=g_key)

In [8]:
locations = citydata[['Lat', 'Long']].astype(float)
humidity = citydata['Humidity']

In [10]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                             dissipating=False, max_intensity=100,
                             point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
#Max Temp: <=75
#Min Temp: >=65
#Windspeed: >=5
#Cloudiness: 0
idealweather = citydata[(citydata['Max Temp'] <= 75)
                        & (citydata['Temperature'] >= 65) 
                        & (citydata['Wind Speed'] >= 5)
                       & (citydata['Cloudiness'] == 0)]
idealweather = idealweather.dropna()
idealweather

,City Number,City,Country,Lat,Long,Humidity,Temperature,Max Temp,Cloudiness,Wind Speed
64,64,Suez,EG,29.97,32.53,70,73.02,73.02,0,11.50
84,84,Angoche,MZ,-16.23,39.91,85,73.26,73.26,0,7.25
179,179,Chalchihuites,MX,23.48,-103.88,26,73.44,73.44,0,9.98
205,205,Noumea,NC,-22.28,166.46,68,68.52,71.01,0,8.05
294,294,Kifrī,IQ,34.70,44.97,22,70.84,70.84,0,7.29
465,465,Port Hedland,AU,-20.32,118.57,88,69.80,69.80,0,10.29
505,505,Concepción del Oro,MX,24.63,-101.42,21,72.95,72.95,0,5.37
506,506,Sambava,MG,-14.27,50.17,82,71.87,71.87,0,10.98
547,547,Tadine,NC,-21.55,167.88,78,72.23,72.23,0,13.73


In [47]:
hotel_df = pd.DataFrame({'Hotel Name': []})


In [70]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'radius': 10000,
    'type': 'lodging',
    'key': g_key
}

for index, row in idealweather.iterrows():
    params['location'] = str(row['Lat'])+','+str(row['Long'])
    city = row['City']
    country = row['Country']
    print(f'Retrieving Results for {city}.')
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest Hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'City'] = city
        hotel_df.loc[index, 'Country'] = country
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print('You goofed')
        
hotel_df

Retrieving Results for Suez.
Closest Hotel in Suez is Green House.
Retrieving Results for Angoche.
Closest Hotel in Angoche is Guest House HESADA.
Retrieving Results for Chalchihuites.
Closest Hotel in Chalchihuites is Hotel Posada Real.
Retrieving Results for Noumea.
Closest Hotel in Noumea is Hôtel Le Lagon.
Retrieving Results for Kifrī.
Closest Hotel in Kifrī is میزان کفری.
Retrieving Results for Port Hedland.
Closest Hotel in Port Hedland is The Esplanade Hotel.
Retrieving Results for Concepción del Oro.
Closest Hotel in Concepción del Oro is HOTEL HACIENDA SAN JAVIER.
Retrieving Results for Sambava.
Closest Hotel in Sambava is Melrose.
Retrieving Results for Tadine.
Closest Hotel in Tadine is Hotel Nengone Village.


,Hotel Name,City,Country,Lat,Lng
64,Green House,Suez,EG,29.960716,32.560655
84,Guest House HESADA,Angoche,MZ,-16.225924,39.905828
179,Hotel Posada Real,Chalchihuites,MX,23.476154,-103.881931
205,Hôtel Le Lagon,Noumea,NC,-22.300991,166.445248
294,میزان کفری,Kifrī,IQ,34.688673,44.958016
465,The Esplanade Hotel,Port Hedland,AU,-20.313925,118.576081
505,HOTEL HACIENDA SAN JAVIER,Concepción del Oro,MX,24.613342,-101.416222
506,Melrose,Sambava,MG,-14.253662,50.157765
547,Hotel Nengone Village,Tadine,NC,-21.599486,167.885646


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [73]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))